In [1]:
import pandas as pd 
import json 
import os 
import numpy as np 

In [2]:
factscore_annotation_path = "../annotations/prod/cliff.jsonl"
df = pd.read_json(factscore_annotation_path, lines=True)
df.head()

,source_id,source,summary,cliff_labels,model,datasource,dataset,qas,spans
0,132,"[Media, playback, is, not, supported, on, this...","[[Laura, Muir, says, she, is, "", gutted, "", sh...","[[world knowledge, correct, correct, correct, ...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 2...","[{'id': 0, 'start': 0, 'end': 2, 'qaIds': [5, ..."
1,132,"[Media, playback, is, not, supported, on, this...","[[Laura, Muir, says, she, is, "", gutted, "", sh...","[[world knowledge, correct, correct, correct, ...",pegasus,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 2...","[{'id': 0, 'start': 0, 'end': 2, 'qaIds': [5, ..."
2,58,"[Trevor, Deely, ,, 22, ,, was, last, seen, wal...","[[Gardaí, (, Irish, police, ), investigating, ...","[[correct, correct, correct, correct, correct,...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 5...","[{'id': 0, 'start': 2, 'end': 4, 'qaIds': [7, ..."
3,58,"[Trevor, Deely, ,, 22, ,, was, last, seen, wal...","[[Police, in, the, Republic, of, Ireland, have...","[[correct, correct, correct, correct, correct,...",pegasus,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 7...","[{'id': 1, 'start': 3, 'end': 6, 'qaIds': [10,..."
4,20,"[The, boy, ,, Julian, Hernandez, ,, disappeare...","[[The, father, of, an, Alabama, boy, who, went...","[[correct, correct, correct, correct, correct,...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 7...","[{'id': 2, 'start': 4, 'end': 5, 'qaIds': [8],..."


In [3]:
def get_individual_score(qa):
    return False if sum(qa["annotations"]) >= 2 else True 

def get_score(qas):
    return np.average([get_individual_score(qa) for qa in qas])

def good_example(qas):
    df_qas = pd.DataFrame(qas)
    df_qas["majority"] = df_qas["annotations"].apply(lambda x: 1 if sum(x) >= 2 else 0)
    flag = False
    for sent_id, sent_qas in df_qas.groupby("sent_id"):
        if sent_qas["majority"].nunique() == 2:
            flag = True
    return flag
    
def is_intrinsic(lst):
    return "intrinsic" in lst[0]

def get_iaa(lst):
    numerator = 0
    for qa in lst:
        if sum(qa["annotations"]) in {0, 3}:
            numerator += 1
    return numerator / len(lst)

df["is_intrinsic"] = df["cliff_labels"].apply(is_intrinsic)
df["score"] = df["qas"].apply(get_score)
df["iaa"] = df["qas"].apply(get_iaa)

In [90]:
df["good"] = df["qas"].apply(good_example)

In [95]:
df[(df["score"] > 0.3) & (df["is_intrinsic"])]

,source_id,source,summary,cliff_labels,model,datasource,dataset,qas,spans,score,good,is_intrinsic,iaa
8,118,"[Mr, Gustafsson, ,, 42, ,, is, being, flown, b...","[[Sweden, 's, former, foreign, minister, Johan...","[[correct, correct, intrinsic, intrinsic, intr...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 1...","[{'id': 0, 'start': 0, 'end': 7, 'qaIds': [0, ...",0.500000,True,True,0.666667
10,17,"[We, are, aware, that, this, interactive, vide...","[[The, way, we, buy, things, has, changed, dra...","[[correct, correct, correct, correct, correct,...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 3...","[{'id': 2, 'start': 2, 'end': 3, 'qaIds': [0],...",0.636364,True,True,0.636364
12,8,"[Gareth, Colfer, -, Williams, ,, 25, ,, died, ...","[[An, inquest, into, the, death, of, a, man, w...","[[correct, correct, correct, correct, correct,...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 9...","[{'id': 5, 'start': 7, 'end': 8, 'qaIds': [10,...",0.450000,True,True,1.000000
13,8,"[Gareth, Colfer, -, Williams, ,, 25, ,, died, ...","[[A, post-mortem, examination, has, failed, to...","[[correct, correct, correct, correct, correct,...",pegasus,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 4...","[{'id': 0, 'start': 1, 'end': 2, 'qaIds': [6],...",0.571429,True,True,1.000000
19,22,"[Pro, -, Kremlin, party, A, Just, Russia, put,...","[[Russia, 's, parliament, is, considering, two...","[[correct, correct, world knowledge, correct, ...",pegasus,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 4...","[{'id': 0, 'start': 0, 'end': 3, 'qaIds': [0],...",0.500000,True,True,0.500000
20,27,"[Investing, Solutions, received, those, benefi...","[[Lettings, agency, Investing, Solutions, has,...","[[correct, correct, correct, correct, correct,...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 6...","[{'id': 3, 'start': 7, 'end': 12, 'qaIds': [7,...",0.625000,True,True,0.750000
22,74,"[About, 80,000, works, of, art, ,, ceramics, ,...","[[The, Wedgwood, Museum, 's, collection, has, ...","[[correct, correct, correct, correct, correct,...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 7...","[{'id': 1, 'start': 1, 'end': 3, 'qaIds': [10]...",0.571429,True,True,0.714286
23,74,"[About, 80,000, works, of, art, ,, ceramics, ,...","[[The, Wedgwood, Museum, 's, collection, has, ...","[[correct, correct, correct, correct, correct,...",pegasus,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 7...","[{'id': 0, 'start': 1, 'end': 3, 'qaIds': [2],...",0.333333,True,True,1.000000
30,71,"[The, casualty, ,, who, was, with, a, group, o...","[[A, paraglider, has, been, airlifted, to, hos...","[[correct, correct, correct, correct, correct,...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 4...","[{'id': 0, 'start': 1, 'end': 2, 'qaIds': [9, ...",0.692308,True,True,0.769231
36,127,"[Thousands, attended, the, early, morning, ser...","[[Hundreds, of, people, have, attended, servic...","[[intrinsic, correct, correct, correct, correc...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 4...","[{'id': 0, 'start': 0, 'end': 3, 'qaIds': [0, ...",0.750000,True,True,1.000000


In [111]:
" ".join(df.iloc[30]["source"])

'The casualty , who was with a group of paragliders , was airlifted off Slieve Gullion at about 15:00 GMT on Sunday . Mourne Mountain Rescue Team said the man had a " serious leg injury " and was taken to the Royal Victoria Hospital in Belfast . The team had been training in the Annalong Valley when the emergency call came at 13:43 GMT . Mourne Mountain Rescue Team , Belfast Coastguard , Irish coastguard , PSNI and the Northern Ireland Ambulance Service were all involved in the rescue . " A total of 22 members responded and the team stood down at 18:17 GMT , " Mourne Mountain Rescue said .'

In [108]:
df.iloc[30]

source_id                                                      71
source          [The, casualty, ,, who, was, with, a, group, o...
summary         [[A, paraglider, has, been, airlifted, to, hos...
cliff_labels    [[correct, correct, correct, correct, correct,...
model                                                        bart
datasource                                                   xsum
dataset                                                     cliff
qas             [{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 4...
spans           [{'id': 0, 'start': 1, 'end': 2, 'qaIds': [9, ...
score                                                    0.692308
good                                                         True
is_intrinsic                                                 True
iaa                                                      0.769231
Name: 30, dtype: object

In [110]:
from itertools import chain
" ".join(list(chain.from_iterable(df.iloc[30]["summary"])))

'A paraglider has been airlifted to hospital with a serious leg injury after falling in the Annalong Valley in County Antrim .'

In [109]:
df.iloc[30]["qas"]

[{'qa_id': 0,
  'sent_id': 0,
  'predicate_idx': 4,
  'predicate_pos': 'VERB',
  'predicate': 'airlifted',
  'question': 'who was airlifted somewhere?',
  'answer_idx': '0-2',
  'answer': 'A paraglider',
  'annotations': [0, 0, 0]},
 {'qa_id': 1,
  'sent_id': 0,
  'predicate_idx': 4,
  'predicate_pos': 'VERB',
  'predicate': 'airlifted',
  'question': 'where was someone airlifted?',
  'answer_idx': '5-7',
  'answer': 'to hospital',
  'annotations': [0, 0, 0]},
 {'qa_id': 2,
  'sent_id': 0,
  'predicate_idx': 4,
  'predicate_pos': 'VERB',
  'predicate': 'airlifted',
  'question': 'why was someone airlifted somewhere?',
  'answer_idx': '7-12',
  'answer': 'with a serious leg injury',
  'annotations': [0, 0, 0]},
 {'qa_id': 3,
  'sent_id': 0,
  'predicate_idx': 4,
  'predicate_pos': 'VERB',
  'predicate': 'airlifted',
  'question': 'when was someone airlifted somewhere?',
  'answer_idx': '12-21',
  'answer': 'after falling in the Annalong Valley in County Antrim',
  'annotations': [1, 1, 

In [26]:
def is_intrinsic(lst):
    return "intrinsic" in lst[0]

def get_iaa(lst):
    numerator = 0
    for qa in lst:
        if sum(qa["annotations"]) in {0, 3}:
            numerator += 1
    return numerator / len(lst)

df["is_intrinsic"] = df["cliff_labels"].apply(is_intrinsic)
df["score"] = df["qas"].apply(get_iaa)
df[df["is_intrinsic"] == True].shape

(21, 11)

In [27]:
df[df["is_intrinsic"] == True]

,source_id,source,summary,cliff_labels,model,datasource,dataset,qas,spans,is_intrinsic,score
0,132,"[Media, playback, is, not, supported, on, this...","[[Laura, Muir, says, she, is, "", gutted, "", sh...","[[world knowledge, correct, correct, correct, ...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 2...","[{'id': 0, 'start': 0, 'end': 2, 'qaIds': [5, ...",True,0.900000
8,118,"[Mr, Gustafsson, ,, 42, ,, is, being, flown, b...","[[Sweden, 's, former, foreign, minister, Johan...","[[correct, correct, intrinsic, intrinsic, intr...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 1...","[{'id': 0, 'start': 0, 'end': 7, 'qaIds': [0, ...",True,0.666667
9,118,"[Mr, Gustafsson, ,, 42, ,, is, being, flown, b...","[[Former, Swedish, Foreign, Minister, Johan, G...","[[intrinsic, intrinsic, intrinsic, intrinsic, ...",pegasus,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 8...","[{'id': 0, 'start': 0, 'end': 6, 'qaIds': [2, ...",True,0.800000
10,17,"[We, are, aware, that, this, interactive, vide...","[[The, way, we, buy, things, has, changed, dra...","[[correct, correct, correct, correct, correct,...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 3...","[{'id': 2, 'start': 2, 'end': 3, 'qaIds': [0],...",True,0.636364
12,8,"[Gareth, Colfer, -, Williams, ,, 25, ,, died, ...","[[An, inquest, into, the, death, of, a, man, w...","[[correct, correct, correct, correct, correct,...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 9...","[{'id': 5, 'start': 7, 'end': 8, 'qaIds': [10,...",True,1.000000
13,8,"[Gareth, Colfer, -, Williams, ,, 25, ,, died, ...","[[A, post-mortem, examination, has, failed, to...","[[correct, correct, correct, correct, correct,...",pegasus,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 4...","[{'id': 0, 'start': 1, 'end': 2, 'qaIds': [6],...",True,1.000000
19,22,"[Pro, -, Kremlin, party, A, Just, Russia, put,...","[[Russia, 's, parliament, is, considering, two...","[[correct, correct, world knowledge, correct, ...",pegasus,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 4...","[{'id': 0, 'start': 0, 'end': 3, 'qaIds': [0],...",True,0.500000
20,27,"[Investing, Solutions, received, those, benefi...","[[Lettings, agency, Investing, Solutions, has,...","[[correct, correct, correct, correct, correct,...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 6...","[{'id': 3, 'start': 7, 'end': 12, 'qaIds': [7,...",True,0.750000
22,74,"[About, 80,000, works, of, art, ,, ceramics, ,...","[[The, Wedgwood, Museum, 's, collection, has, ...","[[correct, correct, correct, correct, correct,...",bart,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 7...","[{'id': 1, 'start': 1, 'end': 3, 'qaIds': [10]...",True,0.714286
23,74,"[About, 80,000, works, of, art, ,, ceramics, ,...","[[The, Wedgwood, Museum, 's, collection, has, ...","[[correct, correct, correct, correct, correct,...",pegasus,xsum,cliff,"[{'qa_id': 0, 'sent_id': 0, 'predicate_idx': 7...","[{'id': 0, 'start': 1, 'end': 3, 'qaIds': [2],...",True,1.000000


In [4]:
df_chatgpt = pd.read_json("../src/data/factscore_origin/labeled/ChatGPT.jsonl", lines=True)
df_chatgpt.head()

,input,output,topic,cat,annotations
0,Question: Tell me a bio of Lanny Flaherty.,Lanny Flaherty is an American actor born on De...,Lanny Flaherty,"[very rare, North America]",[{'text': 'Lanny Flaherty is an American actor...
1,Question: Tell me a bio of Marianne McAndrew.,Marianne McAndrew is an American actress and s...,Marianne McAndrew,"[very rare, North America]",[{'text': 'Marianne McAndrew is an American ac...
2,Question: Tell me a bio of Chaim Malinowitz.,"I'm sorry, but I cannot find any reliable info...",Chaim Malinowitz,"[very rare, North America]",None
3,Question: Tell me a bio of Doug Sheehan.,Doug Sheehan is an American actor who was born...,Doug Sheehan,"[very rare, North America]",[{'text': 'Doug Sheehan is an American actor w...
4,Question: Tell me a bio of Gerhard Fischer (in...,Gerhard Fischer is an inventor and entrepreneu...,Gerhard Fischer (inventor),"[very rare, North America]",[{'text': 'Gerhard Fischer is an inventor and ...


In [6]:
df_chatgpt.iloc[0]["annotations"][0]

{'text': 'Lanny Flaherty is an American actor born on December 18, 1949, in Pensacola, Florida.',
 'is-relevant': True,
 'model-atomic-facts': [{'text': 'Lanny Flaherty is an American actor.'},
  {'text': 'Lanny Flaherty was born on December 18, 1949.'},
  {'text': 'Lanny Flaherty was born in Pensacola, Florida.'}],
 'human-atomic-facts': [{'text': 'Lanny Flaherty is an American.',
   'label': 'S'},
  {'text': 'Lanny Flaherty is an actor.', 'label': 'S'},
  {'text': 'Lanny Flaherty was born on December 18, 1949.', 'label': 'NS'},
  {'text': 'Lanny Flaherty was born in Pensacola.', 'label': 'NS'},
  {'text': 'Pensacola is in Florida.', 'label': 'IR'}]}

In [7]:
factscore_origin = pd.DataFrame()
dir_path = "../src/data/factscore_origin/labeled"
for file in os.listdir(dir_path):
    path = os.path.join(dir_path, file)
    if not path.endswith("jsonl"):
        continue
    df_tmp = pd.read_json(path, lines=True)
    factscore_origin = pd.concat([factscore_origin, df_tmp])

In [9]:
factscore_origin.head()

,input,output,topic,cat,annotations
0,Question: Tell me a bio of Doug Sheehan.,Doug Sheehan is an American actor best known f...,Doug Sheehan,"[very rare, North America]",[{'text': 'Doug Sheehan is an American actor b...
1,Question: Tell me a bio of Taral Hicks.,Taral Hicks is an American singer and actress....,Taral Hicks,"[very rare, North America]",[{'text': 'Taral Hicks is an American singer a...
2,Question: Tell me a bio of Chief Jones.,Chief Jones is an experienced leader with a lo...,Chief Jones,"[very rare, North America]",[{'text': 'Chief Jones is an experienced leade...
3,Question: Tell me a bio of Joey D. Vieira.,"Joey D. Vieira is an award-winning writer, dir...",Joey D. Vieira,"[very rare, North America]",[{'text': 'Joey D. Vieira is an award-winning ...
4,Question: Tell me a bio of Lanny Flaherty.,"Lanny Flaherty (born October 2, 1973) is an Am...",Lanny Flaherty,"[very rare, North America]","[{'text': 'Lanny Flaherty (born October 2, 197..."
